In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
import torch
print("pytorch ver . ",torch.__version__)
print("Is CUDA available?",torch.cuda.is_available())
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import torch.utils.data as Data
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import xarray as xr
import os
os.environ['R_HOME'] = '/home/luhung3080/miniconda3/envs/chou/lib/R'
from rpy2.robjects import r, numpy2ri
numpy2ri.activate()
from rpy2.robjects.packages import importr
sinkr = importr('sinkr')
import Nio
print ("pynio ver . ",Nio.__version__)

pytorch ver .  1.11.0+cu113
Is CUDA available? True
pynio ver .  1.5.5


In [5]:
data=pd.read_csv('/home/luhung3080/Desktop/PycharmProjects/NCHUproject/Transformer/data_final.csv')
data

,SITE_ID,date,FCST_TIME,TAU,pm25_cal,pm25_obs
0,EPA001,2020-02-24 08:00:00,2020-02-24 09:00:00,1,4.9510,10.0
1,EPA001,2020-02-24 08:00:00,2020-02-24 10:00:00,2,4.4674,13.0
2,EPA001,2020-02-24 08:00:00,2020-02-24 11:00:00,3,4.6159,11.0
3,EPA001,2020-02-24 08:00:00,2020-02-24 12:00:00,4,3.9937,11.0
4,EPA001,2020-02-24 08:00:00,2020-02-24 13:00:00,5,3.9602,9.0
...,...,...,...,...,...,...
3092755,EPA080,2021-10-30 08:00:00,2021-11-02 04:00:00,68,3.6190,4.0
3092756,EPA080,2021-10-30 08:00:00,2021-11-02 05:00:00,69,3.7908,7.0
3092757,EPA080,2021-10-30 08:00:00,2021-11-02 06:00:00,70,4.0454,7.0
3092758,EPA080,2021-10-30 08:00:00,2021-11-02 07:00:00,71,3.9015,4.0


# Y = b0 + b1X1(96hr)

In [6]:
###
#cal_PMf  
###
u=np.zeros([544,5112])
for i in range (0,544):
    a=np.array(data['pm25_cal'][5112*i:5112*i+5112])
    u[i]=a.T

###
#obs_PMf
###
v=np.zeros([544,5112])
for i in range (0,544):
    a=np.array(data['pm25_obs'][5112*i:5112*i+5112])
    v[i]=a.T

print(np.shape(u))
print(np.shape(v))

(544, 5112)
(544, 5112)


In [7]:
XRestruct_Fun=u
YRestruct_Fun=v
print(np.shape(XRestruct_Fun))
print(np.shape(YRestruct_Fun))

(544, 5112)
(544, 5112)


In [8]:
Xhat=XRestruct_Fun
Yhat=YRestruct_Fun
print('Xhat')
#print(Xhat)
print(np.shape(Xhat))
print('Yhat')
#print(Yhat)
print(np.shape(Yhat))

Xhat
(544, 5112)
Yhat
(544, 5112)


## PCA

In [9]:
pca95 = PCA(n_components=0.95)
pca95.fit(Xhat)

PCA(n_components=0.95)

In [10]:
x_pca95=pca95.transform(Xhat)
print(x_pca95.shape)
print(np.linalg.matrix_rank(x_pca95))

(544, 157)
157


### LinearRegression


In [11]:
model = LinearRegression()
model.fit(x_pca95, Yhat)
print(np.shape(model.coef_))

(5112, 157)


In [12]:
fit95=model.predict(x_pca95)
print(fit95.shape)

(544, 5112)


### compare (training data)

In [13]:
data0=pd.read_csv('/home/luhung3080/Desktop/PycharmProjects/Data/newdata/result.csv')
dd = data0.drop(data0[data0['TAU'] == 0].index)

In [14]:
fit95_test=fit95
"""
old data
"""
old_x=np.zeros([605,5112])
old_y=np.zeros([605,5112])
#data_cal
for i in range (0,605):
    a=np.array(dd['VALUE'][5112*i:5112*i+5112])
    for j in range(0,5112):
        if a[j]=='NaN' :
            a[j]=np.nan
    old_x[i]=a
#data_obs
for i in range (0,605):
    a=np.array(dd['OBS_VALUE'][5112*i:5112*i+5112])
    for j in range(0,5112):
        if a[j]=='NaN' :
            a[j]=np.nan
        if a[j]=='A' :
            a[j]=np.nan
        if a[j]=='*' :
            a[j]=np.nan
        if a[j]=='x' :
            a[j]=np.nan  
    old_y[i]=a
print(np.shape(old_x))
print(np.shape(old_y))
print(np.shape(fit95_test))

(605, 5112)
(605, 5112)
(544, 5112)


In [17]:
old_x1=np.zeros([544,5112])
old_y1=np.zeros([544,5112])
fit_fx1=np.zeros([544,5112])
for i in range (0,544):
    for j in range (0,5112):
        old_x1[i][j]=old_x[i][j]
        old_y1[i][j]=old_y[i][j]
        fit_fx1[i][j]=fit95_test[i][j]
before1=abs(old_x1-old_y1)
after1=abs(fit_fx1-old_y1)
before_error1=np.nanmean(before1)
after_error1=np.nanmean(after1)
print("January to August (training data set) before_error (MADE) of 95 : %f" %before_error1)
print("January to August (training data set) after_error (MADE) of 95 : %f" %after_error1)

January to August (training data set) before_error (MADE) of 95 : 7.181135
January to August (training data set) after_error (MADE) of 95 : 4.098686


### compare (testing data)

In [19]:
"""
generate new cal
"""
xf=np.zeros([61,5112])
for i in range (544,605):
    a=np.array(data['pm25_cal'][5112*i:5112*i+5112])
    xf[i-544]=a
new_xcal=pca95.transform(xf)
new_xfit=model.predict(new_xcal)

In [20]:
"""
old data
"""
old_x=np.zeros([605,5112])
old_y=np.zeros([605,5112])
#data_cal
for i in range (0,605):
    a=np.array(dd['VALUE'][5112*i:5112*i+5112])
    for j in range(0,5112):
        if a[j]=='NaN' :
            a[j]=np.nan
    old_x[i]=a
#data_obs
for i in range (0,605):
    a=np.array(dd['OBS_VALUE'][5112*i:5112*i+5112])
    for j in range(0,5112):
        if a[j]=='NaN' :
            a[j]=np.nan
        if a[j]=='A' :
            a[j]=np.nan
        if a[j]=='*' :
            a[j]=np.nan
        if a[j]=='x' :
            a[j]=np.nan  
    old_y[i]=a
print(np.shape(old_x))
print(np.shape(old_y))

(605, 5112)
(605, 5112)


In [21]:
old_x2=np.zeros([61,5112])
old_y2=np.zeros([61,5112])
fit_fx2=np.zeros([61,5112])
for i in range (0,61):
    for j in range (0,5112):
        old_x2[i][j]=old_x[i+544][j]
        old_y2[i][j]=old_y[i+544][j]
        fit_fx2[i][j]=new_xfit[i][j]
before2=abs(old_x2-old_y2)
after2=abs(fit_fx2-old_y2)
before_error2=np.nanmean(before2)
after_error2=np.nanmean(after2)
print("September (testing data set) before_error (MADE) of 95 : %f" %before_error2)
print("September (testing data set) after_error (MADE) of 95 : %f" %after_error2)

September (testing data set) before_error (MADE) of 95 : 6.928633
September (testing data set) after_error (MADE) of 95 : 6.480859


In [22]:
#test
per = np.zeros(99)
for i in range (len(per)):
    per[i]=0.01*(i+1)
print(per)
per_error = np.zeros(99)

[0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13 0.14
 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28
 0.29 0.3  0.31 0.32 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42
 0.43 0.44 0.45 0.46 0.47 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56
 0.57 0.58 0.59 0.6  0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7
 0.71 0.72 0.73 0.74 0.75 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84
 0.85 0.86 0.87 0.88 0.89 0.9  0.91 0.92 0.93 0.94 0.95 0.96 0.97 0.98
 0.99]


In [23]:
for z in range (0,99):    
    pca95 = PCA(n_components=per[z])
    pca95.fit(Xhat)
    x_pca95=pca95.transform(Xhat)
    model = LinearRegression()
    model.fit(x_pca95, Yhat)
    print(np.shape(model.coef_))
    fit95=model.predict(x_pca95)
    print(fit95.shape)
    fit95_test=fit95
    """
    old data
    """
    old_x=np.zeros([605,5112])
    old_y=np.zeros([605,5112])
    #data_cal
    for i in range (0,605):
        a=np.array(dd['VALUE'][5112*i:5112*i+5112])
        for j in range(0,5112):
            if a[j]=='NaN' :
                a[j]=np.nan
        old_x[i]=a
    #data_obs
    for i in range (0,605):
        a=np.array(dd['OBS_VALUE'][5112*i:5112*i+5112])
        for j in range(0,5112):
            if a[j]=='NaN' :
                a[j]=np.nan
            if a[j]=='A' :
                a[j]=np.nan
            if a[j]=='*' :
                a[j]=np.nan
            if a[j]=='x' :
                a[j]=np.nan  
        old_y[i]=a
    old_x1=np.zeros([544,5112])
    old_y1=np.zeros([544,5112])
    fit_fx1=np.zeros([544,5112])
    for i in range (0,544):
        for j in range (0,5112):
            old_x1[i][j]=old_x[i][j]
            old_y1[i][j]=old_y[i][j]
            fit_fx1[i][j]=fit95_test[i][j]
    before1=abs(old_x1-old_y1)
    after1=abs(fit_fx1-old_y1)
    before_error1=np.nanmean(before1)
    after_error1=np.nanmean(after1)

    """
    generate new cal
    """
    xf=np.zeros([61,5112])
    for i in range (544,605):
        a=np.array(data['pm25_cal'][5112*i:5112*i+5112])
        xf[i-544]=a
    new_xcal=pca95.transform(xf)
    new_xfit=model.predict(new_xcal)

    """
    old data
    """
    old_x=np.zeros([605,5112])
    old_y=np.zeros([605,5112])
    #data_cal
    for i in range (0,605):
        a=np.array(dd['VALUE'][5112*i:5112*i+5112])
        for j in range(0,5112):
            if a[j]=='NaN' :
                a[j]=np.nan
        old_x[i]=a
    #data_obs
    for i in range (0,605):
        a=np.array(dd['OBS_VALUE'][5112*i:5112*i+5112])
        for j in range(0,5112):
            if a[j]=='NaN' :
                a[j]=np.nan
            if a[j]=='A' :
                a[j]=np.nan
            if a[j]=='*' :
                a[j]=np.nan
            if a[j]=='x' :
                a[j]=np.nan  
        old_y[i]=a

    old_x2=np.zeros([61,5112])
    old_y2=np.zeros([61,5112])
    fit_fx2=np.zeros([61,5112])
    for i in range (0,61):
        for j in range (0,5112):
            old_x2[i][j]=old_x[i+544][j]
            old_y2[i][j]=old_y[i+544][j]
            fit_fx2[i][j]=new_xfit[i][j]
    before2=abs(old_x2-old_y2)
    after2=abs(fit_fx2-old_y2)
    before_error2=np.nanmean(before2)
    after_error2=np.nanmean(after2)
    print("January to August (training data set) before_error (MADE) of 95 : %f" %before_error1)
    print("January to August (training data set) after_error (MADE) of 95 : %f" %after_error1)
    print("September (testing data set) before_error (MADE) of 95 : %f" %before_error2)
    print("September (testing data set) after_error (MADE) of 95 : %f" %after_error2)
    per_error[z] = after_error2

(5112, 1)
(544, 5112)
January to August (training data set) before_error (MADE) of 95 : 7.181135
January to August (training data set) after_error (MADE) of 95 : 6.294874
September (testing data set) before_error (MADE) of 95 : 6.928633
September (testing data set) after_error (MADE) of 95 : 7.249428
(5112, 1)
(544, 5112)
January to August (training data set) before_error (MADE) of 95 : 7.181135
January to August (training data set) after_error (MADE) of 95 : 6.294874
September (testing data set) before_error (MADE) of 95 : 6.928633
September (testing data set) after_error (MADE) of 95 : 7.249428
(5112, 1)
(544, 5112)
January to August (training data set) before_error (MADE) of 95 : 7.181135
January to August (training data set) after_error (MADE) of 95 : 6.294874
September (testing data set) before_error (MADE) of 95 : 6.928633
September (testing data set) after_error (MADE) of 95 : 7.249428
(5112, 1)
(544, 5112)
January to August (training data set) before_error (MADE) of 95 : 7.1811

(5112, 1)
(544, 5112)
January to August (training data set) before_error (MADE) of 95 : 7.181135
January to August (training data set) after_error (MADE) of 95 : 6.294874
September (testing data set) before_error (MADE) of 95 : 6.928633
September (testing data set) after_error (MADE) of 95 : 7.249428
(5112, 1)
(544, 5112)
January to August (training data set) before_error (MADE) of 95 : 7.181135
January to August (training data set) after_error (MADE) of 95 : 6.294874
September (testing data set) before_error (MADE) of 95 : 6.928633
September (testing data set) after_error (MADE) of 95 : 7.249428
(5112, 1)
(544, 5112)
January to August (training data set) before_error (MADE) of 95 : 7.181135
January to August (training data set) after_error (MADE) of 95 : 6.294874
September (testing data set) before_error (MADE) of 95 : 6.928633
September (testing data set) after_error (MADE) of 95 : 7.249428
(5112, 1)
(544, 5112)
January to August (training data set) before_error (MADE) of 95 : 7.1811

(5112, 4)
(544, 5112)
January to August (training data set) before_error (MADE) of 95 : 7.181135
January to August (training data set) after_error (MADE) of 95 : 5.698209
September (testing data set) before_error (MADE) of 95 : 6.928633
September (testing data set) after_error (MADE) of 95 : 6.921050
(5112, 4)
(544, 5112)
January to August (training data set) before_error (MADE) of 95 : 7.181135
January to August (training data set) after_error (MADE) of 95 : 5.698209
September (testing data set) before_error (MADE) of 95 : 6.928633
September (testing data set) after_error (MADE) of 95 : 6.921050
(5112, 5)
(544, 5112)
January to August (training data set) before_error (MADE) of 95 : 7.181135
January to August (training data set) after_error (MADE) of 95 : 5.647814
September (testing data set) before_error (MADE) of 95 : 6.928633
September (testing data set) after_error (MADE) of 95 : 6.567329
(5112, 5)
(544, 5112)
January to August (training data set) before_error (MADE) of 95 : 7.1811

(5112, 40)
(544, 5112)
January to August (training data set) before_error (MADE) of 95 : 7.181135
January to August (training data set) after_error (MADE) of 95 : 5.080592
September (testing data set) before_error (MADE) of 95 : 6.928633
September (testing data set) after_error (MADE) of 95 : 6.352470
(5112, 45)
(544, 5112)
January to August (training data set) before_error (MADE) of 95 : 7.181135
January to August (training data set) after_error (MADE) of 95 : 5.004157
September (testing data set) before_error (MADE) of 95 : 6.928633
September (testing data set) after_error (MADE) of 95 : 6.461410
(5112, 51)
(544, 5112)
January to August (training data set) before_error (MADE) of 95 : 7.181135
January to August (training data set) after_error (MADE) of 95 : 4.941045
September (testing data set) before_error (MADE) of 95 : 6.928633
September (testing data set) after_error (MADE) of 95 : 6.490987
(5112, 58)
(544, 5112)
January to August (training data set) before_error (MADE) of 95 : 7.

In [24]:
print(per_error)

[7.24942835 7.24942835 7.24942835 7.24942835 7.24942835 7.24942835
 7.24942835 7.24942835 7.24942835 7.24942835 7.24942835 7.24942835
 7.24942835 7.24942835 7.24942835 7.24942835 7.24942835 7.24942835
 7.24942835 7.24942835 7.24942835 7.24942835 7.24942835 7.24942835
 7.24942835 7.24942835 7.24942835 7.24942835 7.24942835 7.24942835
 7.24942835 7.24942835 7.24942835 7.24942835 7.24942835 7.24942835
 7.24942835 7.27598043 7.27598043 7.27598043 7.27598043 7.27598043
 7.27598043 7.27598043 7.27598043 7.27598043 7.27598043 7.27598043
 7.27598043 7.27598043 7.24673273 7.24673273 7.24673273 7.24673273
 7.24673273 7.24673273 6.92104963 6.92104963 6.567329   6.567329
 6.567329   6.52826873 6.52826873 6.52826873 6.5363984  6.5363984
 6.55666778 6.56834936 6.56834936 6.50331392 6.49322022 6.48831343
 6.48608337 6.41704139 6.43480476 6.38132751 6.24655463 6.28689986
 6.26565093 6.26172256 6.2806103  6.26467336 6.30293906 6.30905464
 6.3524704  6.46140981 6.49098661 6.52406675 6.41680256 6.3504059

In [25]:
print(0.01*(np.argmin(per_error)+1))
print(min(per_error))

0.77
6.246554634591224
